In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0411 21:34:47.080355053     973 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0411 21:34:51.400946221     973 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
shn = gpd.read_parquet(f'{utils.BUS_GCS_FILE_PATH}state_highway_network.parquet')

In [18]:
shn_proj = shn.to_crs(shared_utils.geography_utils.CA_NAD83Albers)
shn_proj.geometry = shn_proj.buffer(40)

In [19]:
shn_to_map = shn_proj.to_crs(shared_utils.geography_utils.WGS84)

In [3]:
## parameters cell
itp_id = 300

In [4]:
analysis_date = dt.date(2022, 3, 30) ##TODO shift to Mar 30 for all?

In [5]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_03_30.parquet')

In [6]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_03_30.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [8]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [9]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [20]:
rt_day.shn_gdf = shn_to_map

In [21]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [22]:
m = rt_day.segment_speed_map(how='low_speeds')

speed above 80 for shape 25944, dropping
negative speed for shape 25920, dropping
speed above 80 for shape 25877, dropping


/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py:243: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [1]:
# m

In [2]:
def _show_speed_map(self, how = 'average',
                      colorscale = ZERO_THIRTY_COLORSCALE, size = [900, 550]):

    gdf = self.stop_segment_speed_view.copy()
    # gdf = self._filter(gdf)
    # print(gdf.dtypes)
    singletrip = gdf.trip_id.nunique() == 1
    gdf = gdf >> distinct(_.shape_id, _.stop_sequence, _keep_all=True) ## essential here for reasonable map size!
    orig_rows = gdf.shape[0]
    gdf = gdf.round({'avg_mph': 1, '_20p_mph': 1, 'shape_meters': 0,
                    'trips_per_hour': 1}) ##round for display

    how_speed_col = {'average': 'avg_mph', 'low_speeds': '_20p_mph'}
    how_formatted = {'average': 'Average', 'low_speeds': '20th Percentile'}

    gdf = gdf >> select(-_.service_date)
    gdf = gdf >> arrange(_.trips_per_hour)
    gdf = gdf.set_crs(CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(try_parallel)
    self.detailed_map_view = gdf.copy()
    ## create clips, integrate buffer+simplify?
    gdf.geometry = gdf.geometry.apply(arrowize_segment).simplify(tolerance=5)
    gdf = gdf >> filter(gdf.geometry.is_valid)
    gdf = gdf >> filter(-gdf.geometry.is_empty)

    assert gdf.shape[0] >= orig_rows*.99, 'over 1% of geometries invalid after buffer+simplify'
    gdf = gdf.to_crs(WGS84)
    centroid = gdf.dissolve().centroid 
    name = self.calitp_agency_name

    popup_dict = {
        how_speed_col[how]: "Speed (miles per hour)",
        "shape_meters": "Distance along route (meters)",
        "route_short_name": "Route",
        "shape_id": "Shape ID",
        "direction_id": "Direction ID",
        "stop_id": "Next Stop ID",
        "stop_sequence": "Next Stop Sequence",
        "trips_per_hour": "Trips per Hour" 
    }
    if singletrip:
        popup_dict["delay_seconds"] = "Current Delay (seconds)"
        popup_dict["delay_chg_sec"] = "Change in Delay (seconds)"

    # g = make_folium_choropleth_map(
    #     gdf,
    #     plot_col = how_speed_col[how],
    #     popup_dict = popup_dict,
    #     tooltip_dict = popup_dict,
    #     colorscale = colorscale,
    #     fig_width = size[0], fig_height = size[1],
    #     zoom = 13,
    #     centroid = [centroid.y, centroid.x],
    #     title=f"{name} {how_formatted[how]} Vehicle Speeds Between Stops{self.filter_formatted}",
    #     legend_name = "Speed (miles per hour)",
    #     highlight_function=lambda x: {
    #         'fillColor': '#DD1C77',
    #         "fillOpacity": 0.6,
    #     }
    # )
    shn_dict = {'Route': 'Route', 'District': 'District'}

    LAYERS_DICT = {
        "Bus Speeds": {'df': gdf,
                    'plot_col': how_speed_col[how],
                    'popup_dict': popup_dict,
                    'tooltip_dict': popup_dict,
                    'colorscale': colorscale, 
        },
        "State Highway Network": {'df': self.shn_gdf,
                                  'plot_col': 'Route',
                                  'popup_dict': shn_dict,
                                  'tooltip_dict': shn_dict,
                                  'colorscale': lambda x: shared_utils.calitp_color_palette.CALITP_CATEGORY_BRIGHT_COLORS[0]
        }
    }

    g = make_folium_multiple_layers_map(
        LAYERS_DICT,
        fig_width = size[0], fig_height = size[1],
        zoom = 13,
        centroid = [centroid.y, centroid.x],
        title=f"{name} {how_formatted[how]} Vehicle Speeds Between Stops{self.filter_formatted}",
        legend_name = "Speed (miles per hour)",
        highlight_function=lambda x: {
            'fillColor': '#DD1C77',
            "fillOpacity": 0.6,
        }
    )

    return g  

NameError: name 'ZERO_THIRTY_COLORSCALE' is not defined